In [1]:
# If you have not done so already: pip install sklearn-crfsuite
import sklearn_crfsuite
#import pandas as pd
import glob
from csv import DictReader, DictWriter

In [2]:
filename_train = './../../Data/parc_features/parc_train_features_small.tsv'
filename_test = './../../Data/parc_features/parc_test_features_small.tsv'
filepath_out = './../../Data/parc_features/parc_test_predicted_features_small.tsv'

In [9]:
# python content_detection_model.py ./../../Data/parc_features/parc_train_features_small.tsv ./../../Data/parc_features/parc_test_features_small.tsv ./../../Data/parc_features/parc_test_predicted_features_small.tsv

SyntaxError: invalid syntax (<ipython-input-9-bb78fd3d9e77>, line 1)

In [3]:
def extract_features_and_labels(list_of_dicts):
    feature_dicts = []
    label_list = []
    for token_dict in list_of_dicts:
        label = token_dict.pop('content_label_gold')
        feature_dicts.append(token_dict)
        label_list.append(label)
        
    return [feature_dicts], [label_list]


def read_data(filepath):
    with open(filepath, 'r') as infile:
        dict_reader = DictReader(infile, delimiter='\t')
        data = list(dict_reader)
    return data

def train_crf(X_train, y_train):
    crf = sklearn_crfsuite.CRF(  
            algorithm='lbfgs',  
            c1=0.1,  
            c2=0.1,  
            max_iterations=100,  
            all_possible_transitions=True  
        ) 
    crf.fit(X_train, y_train)
    return crf

def test_crf_prediction(crf, X_test, y_test):  
    #labels = show_crf_label(crf)  
    y_pred = crf.predict(X_test)  
    return y_pred

def join_test_gold_pred(X_test, y_test, y_pred): #TODO Change to X_test[0]
    assert len(X_test[0]) == len(y_test[0]), 'X_test and y_test are not same length'
    assert len(y_test[0]) == len(y_pred[0]), 'y_test and y_pred are not same length'
    test_pred_data = []
    for i in range(len(X_test[0])): 
        token_dict = X_test[0][i]
        
        token_dict['content_label_gold'] = y_test[0][i]
        token_dict['content_label_pred'] = y_pred[0][i]
        test_pred_data.append(token_dict)
    return test_pred_data
        
def write_dicts_to_file(filepath_out, list_of_dicts):
    with open(filepath_out, 'w', newline='\n') as outfile:
        writer = DictWriter(outfile, fieldnames=list_of_dicts[0].keys(), delimiter='\t')
        writer.writeheader()
        for data in list_of_dicts:
            writer.writerow(data)

In [7]:
# Read in data
train_data = read_data(filename_train)
test_data = read_data(filename_test)
# Separate data into features and labels
X_train, y_train = extract_features_and_labels(train_data)
X_test, y_test = extract_features_and_labels(test_data)
# Train the model
crf = train_crf(X_train, y_train)
# Predict on test data
y_pred = test_crf_prediction(crf, X_test, y_test)
# Join features, gold labels and predicted labels
true_pred_data = join_test_gold_pred(X_test, y_test, y_pred)
# Write to file
write_dicts_to_file(filepath_out, true_pred_data)